<a href="https://colab.research.google.com/github/adedoyin-a/water-pollution-prediction/blob/main/AIE_557_Research_Water_Quality_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# install needed libraries
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, precision_score, f1_score, recall_score, classification_report, confusion_matrix

from google.colab import files
uploaded = files.upload()

Saving Beach_Water_Quality_-_Automated_Sensors_20250404.csv to Beach_Water_Quality_-_Automated_Sensors_20250404.csv


In [2]:
df = pd.read_csv('Beach_Water_Quality_-_Automated_Sensors_20250404.csv')
df.head()

,Beach Name,Measurement Timestamp,Water Temperature,Turbidity,Transducer Depth,Wave Height,Wave Period,Battery Life,Measurement Timestamp Label,Measurement ID
0,Calumet Beach,04/07/2018 12:30:00 AM,21.7,3.15,NaN,0.389,3.0,0.0,04/07/2018 12:30 AM,CalumetBeach201804072430
1,Ohio Street Beach,06/14/2019 07:00:00 PM,16.0,2.18,NaN,0.137,3.0,11.8,06/14/2019 7:00 PM,OhioStreetBeach201906141900
2,Ohio Street Beach,05/16/2018 01:00:00 PM,21.7,1.92,NaN,0.023,1.0,10.3,05/16/2018 1:00 PM,OhioStreetBeach201805161300
3,Ohio Street Beach,05/16/2018 02:00:00 PM,21.9,1.81,NaN,0.026,6.0,10.3,05/16/2018 2:00 PM,OhioStreetBeach201805161400
4,Ohio Street Beach,06/04/2018 09:00:00 AM,23.0,0.03,NaN,0.021,8.0,12.4,06/04/2018 9:00 AM,OhioStreetBeach201806040900


In [ ]:
# drop columns that are not needed for RandomForest Classification

df = df.drop(['Measurement Timestamp', 'Measurement Timestamp Label', 'Measurement ID'], axis=1)

# Encode Beach Name

le = LabelEncoder()
df['Beach Name'] = le.fit_transform(df['Beach Name'])

df['Pollution Status'] = df['Turbidity'].apply(lambda x: 'Polluted' if x > 5 else 'Safe')

# Features and labels

X = df.drop(['Pollution Status', 'Turbidity'], axis=1)
y = df['Pollution Status']

# drop rows with missing turbidity value

df = df.dropna(subset=['Turbidity'])

# split dataset into training and testing

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# make prediction

y_pred = model.predict(X_test)



In [6]:
df.head()


,Beach Name,Water Temperature,Turbidity,Transducer Depth,Wave Height,Wave Period,Battery Life,Pollution Status
0,1,21.7,3.15,NaN,0.389,3.0,0.0,Safe
1,4,16.0,2.18,NaN,0.137,3.0,11.8,Safe
2,4,21.7,1.92,NaN,0.023,1.0,10.3,Safe
3,4,21.9,1.81,NaN,0.026,6.0,10.3,Safe
4,4,23.0,0.03,NaN,0.021,8.0,12.4,Safe
